## Classification of Anomalies  in Gastrointestinal Tract through Endoscopic Imagery with Deep Learning

### Transfer learning using InceptionV3 model

#### Download kvasir-dataset-v2

In [ ]:
!wget --no-check-certificate \
  https://datasets.simula.no/kvasir/data/kvasir-dataset-v2.zip\
  -O /tmp/kvasir-dataset-v2.zip

#### Extract Dataset

In [ ]:
import os
import zipfile

zip_file = '/tmp/kvasir-dataset-v2.zip'
zip_ref = zipfile.ZipFile(zip_file, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Dense
from keras.utils import get_file
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

#### Generate data local kvasir-dataset-v2

In [2]:
path ='/tmp/kvasir-dataset-v2'

image_gen = ImageDataGenerator(
    rescale=1/255,fill_mode='nearest', 
    brightness_range= [0.4,1],
    validation_split = 0.2,
    zoom_range=0.2,)

train_generator = image_gen.flow_from_directory(
    path,
    target_size=(224,224),
    batch_size=32,
    subset="training",
    seed=1337,
    class_mode='categorical'
)

validation_generator = image_gen.flow_from_directory(
    path,
    target_size=(224,224),
    batch_size=32,
    subset="validation",
    seed=1337,
    class_mode='categorical'
)

Found 3200 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


#### Instantiates the InceptionV3 architecture

In [ ]:
inception_model = InceptionV3(input_shape=(224, 224, 3),
                                include_top=False,
                                weights='imagenet')

for layer in inception_model.layers[:40]:
        layer.trainable = False
for layer in inception_model.layers[40:]:
        layer.trainable = True

last_layer = inception_model.get_layer('mixed7')
last_output = last_layer.output

#### Adding additional layers and compiling the model

In [ ]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(8, activation='softmax')(x)

model =  tf.keras.Model( inception_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

#### Training model

In [ ]:
epochs = 50

history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 5,
            epochs = epochs,
            validation_steps = 5,
            verbose = 1)

#### Print training and validation accuracy

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()